In [1]:
import requests
import numpy as np
import pandas as pd

from datetime import datetime

from bs4 import BeautifulSoup

In [18]:
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36'}

page = requests.get(url, headers=headers)

soup = BeautifulSoup(page.text, 'html.parser')

products = soup.find('ul', class_='products-listing small')

product_list = products.find_all('article', class_='hm-product-item')

product_id = [p.get('data-articlecode') for p in product_list]
product_category = [p.get('data-category') for p in product_list]

# list = []
# for i in range(len(product_id)):
#     list.append(product_id[i])
#     list.append(product_category[i])
# list

product_list = products.find_all('a', class_='link')
product_name = [p.get_text() for p in product_list]


product_list = products.find_all('span', class_='price regular')
product_price = [p.get_text() for p in product_list]


data = pd.DataFrame( [product_id, product_category, product_name, product_price]).T
data.columns = ['product_id', 'product_category', 'product_name', 'product_price']

data['scrapy_datetime'] = datetime.now().strftime( '%Y-%m-%d %H:%M:%S')

In [19]:
data.head(10)

,product_id,product_category,product_name,product_price,scrapy_datetime
0,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,2022-10-25 17:25:37
1,0985159001,men_jeans_skinny,Skinny Jeans,$ 24.99,2022-10-25 17:25:37
2,1071707008,men_jeans_relaxed,Relaxed Jeans,$ 29.99,2022-10-25 17:25:37
3,1100162002,men_jeans_regular,Essentials No 2: THE JEANS,$ 39.99,2022-10-25 17:25:37
4,1024711006,men_jeans_slim,Slim Jeans,$ 29.99,2022-10-25 17:25:37
5,1013317003,men_jeans_joggers,Hybrid Regular Tapered Joggers,$ 39.99,2022-10-25 17:25:37
6,1008110006,men_jeans_slim,Freefit® Slim Jeans,$ 49.99,2022-10-25 17:25:37
7,1008549003,men_jeans_regular,Regular Jeans,$ 24.99,2022-10-25 17:25:37
8,0993887008,men_jeans_regular,Hybrid Regular Jeans,$ 39.99,2022-10-25 17:25:37
9,0690449067,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-10-25 17:25:37


In [59]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36'}

#empty dataframe
df_details = pd.DataFrame()

aux = []

cols = ['Additional material information', 'Art. No.', 'Care instructions', 'Composition', 'Concept', 'Description',
'Fit', 'Imported', 'Material', 'Nice to know', 'Size', 'messages.clothingStyle', 'messages.garmentLength',
'messages.waistRise']

df_pattern = pd.DataFrame(columns=cols)

for i in range(len(data)):
    # API Requests
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html'
    
    page = requests.get(url, headers=headers)

    #Beautiful Soup object
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # ==================== color name ==================== #
    product_list = soup.find_all('a', class_='filter-option miniature')
    color_name = [p.get('data-color') for p in product_list]
    
    # product id
    product_id = [p.get('data-articlecode') for p in product_list]
    
    df_color = pd.DataFrame( [product_id, color_name] ).T
    df_color.columns = ['product_id', 'color_name']
    
    # generate style id + color id
    df_color['style_id'] = df_color['product_id'].apply(lambda x: x[:-3])
    df_color['color_id'] = df_color['product_id'].apply(lambda x: x[-3:])
    
    
    # ==================== composition ==================== #
    product_composition_list = soup.find_all('div', class_='details-attributes-list-item')
    product_composition = [list(filter(None, p.get_text().split('\n'))) for p in product_composition_list]

    # rename dataframe
    data_composition = pd.DataFrame(product_composition).T
    data_composition.columns = data_composition.iloc[0]
    
    # delete first row    
    data_composition = data_composition.iloc[1:].fillna(method='ffill')
    
    #guarantee the same numnber of columns
    data_composition = pd.concat([df_pattern, data_composition], axis=0)
    
    #generate style id + color id
    data_composition['style_id'] = data_composition['Art. No.'].apply(lambda x: x[:-3])
    data_composition['color_id'] = data_composition['Art. No.'].apply(lambda x: x[-3:])
    
    data_composition = data_composition.drop_duplicates(subset=['Fit', 'Composition', 'Art. No.', 'Size'], keep='first', inplace=False, ignore_index=False)
    
    aux = aux + data_composition.columns.tolist()
    
    # merge data color + decomposition
    data_sku = pd.merge(df_color, data_composition[['style_id', 'Fit', 'Composition', 'Size', 'Care instructions']], how='left', on='style_id')
    
    # all detail products
    df_details = pd.concat( [df_details, data_sku], axis=0)
    
# data_composition = data_composition[['Fit','Composition','Art. No.']]
# data_composition = data_composition.iloc[1:].fillna(method='ffill')   
    


# data_composition = data_composition[['Fit', 'Composition', 'Art. No.', 'style_id', 'color_id']].copy()



In [60]:
df_details.head(15)


,product_id,color_name,style_id,color_id,Fit,Composition,Size,Care instructions
0,1024256002,Light denim blue,1024256,002,Slim fit,"Shell: Cotton 99%, Spandex 1%",NaN,Only non-chlorine bleach when needed
1,1024256002,Light denim blue,1024256,002,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%",NaN,Medium iron
2,1024256003,Light denim blue,1024256,003,Slim fit,"Shell: Cotton 99%, Spandex 1%",NaN,Only non-chlorine bleach when needed
3,1024256003,Light denim blue,1024256,003,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%",NaN,Medium iron
4,1024256004,Denim blue,1024256,004,Slim fit,"Shell: Cotton 99%, Spandex 1%",NaN,Only non-chlorine bleach when needed
5,1024256004,Denim blue,1024256,004,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%",NaN,Medium iron
6,1024256005,Dark blue,1024256,005,Slim fit,"Shell: Cotton 99%, Spandex 1%",NaN,Only non-chlorine bleach when needed
7,1024256005,Dark blue,1024256,005,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%",NaN,Medium iron
8,1024256006,Dark denim blue,1024256,006,Slim fit,"Shell: Cotton 99%, Spandex 1%",NaN,Only non-chlorine bleach when needed
9,1024256006,Dark denim blue,1024256,006,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%",NaN,Medium iron


In [61]:
# Join Showroom data + details
data['style_id'] = data['product_id'].apply(lambda x: x[:-3])
data['color_id'] = data['product_id'].apply(lambda x: x[-3:])
data.head()

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id
0,1024256001,men_jeans_slim,slim_jeans,19.99,2022-10-25 17:25:37,1024256,001
1,0985159001,men_jeans_skinny,skinny_jeans,24.99,2022-10-25 17:25:37,0985159,001
2,1071707008,men_jeans_relaxed,relaxed_jeans,29.99,2022-10-25 17:25:37,1071707,008
3,1100162002,men_jeans_regular,essentials_no_2:_the_jeans,39.99,2022-10-25 17:25:37,1100162,002
4,1024711006,men_jeans_slim,slim_jeans,29.99,2022-10-25 17:25:37,1024711,006


In [62]:
data['product_price'] = data['product_price'].apply(lambda x: x.replace('$ ', '') if pd.notnull(x) else x)

print(data['style_id'].count())
print(df_details['style_id'].count())

data.head()



36
512


,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id
0,1024256001,men_jeans_slim,slim_jeans,19.99,2022-10-25 17:25:37,1024256,001
1,0985159001,men_jeans_skinny,skinny_jeans,24.99,2022-10-25 17:25:37,0985159,001
2,1071707008,men_jeans_relaxed,relaxed_jeans,29.99,2022-10-25 17:25:37,1071707,008
3,1100162002,men_jeans_regular,essentials_no_2:_the_jeans,39.99,2022-10-25 17:25:37,1100162,002
4,1024711006,men_jeans_slim,slim_jeans,29.99,2022-10-25 17:25:37,1024711,006


In [63]:
data['product_name'] = data['product_name'].apply(lambda x: x.replace(' ', '_').lower())
data['scrapy_datetime'] = pd.to_datetime(data['scrapy_datetime'], format='%Y-%m-%d %H:%M:%S')
# data['style_id'] = data['style_id'].astype(int)
# data['color_id'] = data['color_id'].astype(int)

data

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id
0,1024256001,men_jeans_slim,slim_jeans,19.99,2022-10-25 17:25:37,1024256,001
1,0985159001,men_jeans_skinny,skinny_jeans,24.99,2022-10-25 17:25:37,0985159,001
2,1071707008,men_jeans_relaxed,relaxed_jeans,29.99,2022-10-25 17:25:37,1071707,008
3,1100162002,men_jeans_regular,essentials_no_2:_the_jeans,39.99,2022-10-25 17:25:37,1100162,002
4,1024711006,men_jeans_slim,slim_jeans,29.99,2022-10-25 17:25:37,1024711,006
5,1013317003,men_jeans_joggers,hybrid_regular_tapered_joggers,39.99,2022-10-25 17:25:37,1013317,003
6,1008110006,men_jeans_slim,freefit®_slim_jeans,49.99,2022-10-25 17:25:37,1008110,006
7,1008549003,men_jeans_regular,regular_jeans,24.99,2022-10-25 17:25:37,1008549,003
8,0993887008,men_jeans_regular,hybrid_regular_jeans,39.99,2022-10-25 17:25:37,0993887,008
9,0690449067,men_jeans_ripped,skinny_jeans,39.99,2022-10-25 17:25:37,0690449,067


In [89]:
data_raw = pd.merge(data, df_details[['product_id', 'style_id', 'color_name', 'Fit', 'Composition', 'Size']], on='product_id')
print(data_raw['product_id'].count())
del data_raw['style_id_y']
data_raw

216


,product_id,product_category,product_name,product_price,scrapy_datetime,style_id_x,color_id,color_name,Fit,Composition,Size
0,1024256001,men_jeans_slim,slim_jeans,19.99,2022-10-25 17:25:37,1024256,001,Black,Slim fit,"Shell: Cotton 99%, Spandex 1%",NaN
1,1024256001,men_jeans_slim,slim_jeans,19.99,2022-10-25 17:25:37,1024256,001,Black,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%",NaN
2,1024256001,men_jeans_slim,slim_jeans,19.99,2022-10-25 17:25:37,1024256,001,Black,Slim fit,"Shell: Cotton 99%, Spandex 1%",NaN
3,1024256001,men_jeans_slim,slim_jeans,19.99,2022-10-25 17:25:37,1024256,001,Black,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%",NaN
4,1024256001,men_jeans_slim,slim_jeans,19.99,2022-10-25 17:25:37,1024256,001,Black,Slim fit,"Shell: Cotton 99%, Spandex 1%",NaN
...,...,...,...,...,...,...,...,...,...,...,...
211,0979945012,men_jeans_loose,loose_jeans,39.99,2022-10-25 17:25:37,0979945,012,Gray,Loose fit,Shell: Cotton 100%,NaN
212,0979945012,men_jeans_loose,loose_jeans,39.99,2022-10-25 17:25:37,0979945,012,Gray,Loose fit,"Pocket lining: Polyester 65%, Cotton 35%",NaN
213,1024711001,men_jeans_slim,slim_jeans,29.99,2022-10-25 17:25:37,1024711,001,Black,Slim fit,"Shell: Cotton 98%, Spandex 2%",NaN
214,1024711001,men_jeans_slim,slim_jeans,29.99,2022-10-25 17:25:37,1024711,001,Black,Slim fit,Pocket lining: Cotton 100%,NaN


In [93]:
# data_raw[['product_id', 'Composition']].sample(10)
data_raw = data_raw[~data_raw['Composition'].str.contains('Pocket lining:', na=False)]
# data_raw = data_raw[~data_raw['Composition'].str.contains('Pocket:', na=False)]
data_raw = data_raw[~data_raw['Composition'].str.contains('Shell:', na=False)]
data_raw = data_raw[~data_raw['Composition'].str.contains('Lining:', na=False)]
data_raw[['product_id', 'Composition']].sample(25)

,product_id,Composition
33,0690449067,"Cotton 98%, Spandex 2%"
197,0690449059,"Cotton 98%, Spandex 2%"
162,0690449036,"Cotton 98%, Spandex 2%"
20,1013317003,"Cotton 78%, Polyester 21%, Spandex 1%"
37,0690449067,"Cotton 98%, Spandex 2%"
198,0690449059,"Cotton 98%, Spandex 2%"
131,0690449051,"Cotton 98%, Spandex 2%"
82,0690449022,"Cotton 98%, Spandex 2%"
81,0690449022,"Cotton 98%, Spandex 2%"
177,0690449064,"Cotton 98%, Spandex 2%"


In [92]:
data_raw.shape

(25, 11)